In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
import keras
from keras import *
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras_preprocessing.image import ImageDataGenerator
import json

Using TensorFlow backend.


In [3]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://10.137.32.250:8470', zone='us-central1-f')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

tpu_strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: 10.137.32.250:8470


INFO:tensorflow:Initializing the TPU system: 10.137.32.250:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [4]:
image_generator = ImageDataGenerator(rescale=1./255,
                                     zoom_range=0.2,
                                     rotation_range=45,
                                     horizontal_flip=True,
                                     validation_split=0.05)

In [5]:
INPUT_SHAPE = (224, 224, 3)
BATCH_SIZE = 32
EPOCHS = 5

In [6]:
training_set = image_generator.flow_from_directory('../Dataset/train/herbarium/images/',
                                                   target_size=INPUT_SHAPE[:2],
                                                   batch_size=BATCH_SIZE,
                                                   class_mode='categorical',
                                                   subset='training')
validation_set = image_generator.flow_from_directory('../Dataset/train/herbarium/images/',
                                                     target_size=INPUT_SHAPE[:2],
                                                     batch_size=BATCH_SIZE,
                                                     class_mode='categorical',
                                                     subset='validation')

Found 305208 images belonging to 998 classes.
Found 15552 images belonging to 998 classes.


In [7]:
CLASSES = 998

In [8]:
vgg_base = applications.VGG19(include_top=False, input_shape=INPUT_SHAPE, pooling='max', classes=CLASSES)
for layer in vgg_base.layers:
    layer.trainable = False
vgg_base.get_layer('block5_conv4').trainable = True

In [9]:
vgg_output = vgg_base.get_layer('block5_pool').output
top = Flatten()(vgg_output)
top = Dense(units=4096, activation='relu')(top)
top = Dense(units=4096, activation='relu')(top)
top = Dense(units=CLASSES, activation='softmax')(top)

In [10]:
with tpu_strategy.scope():
    model = Model(vgg_base.input, top)
    model.summary()
    model.compile(optimizer=Adam(learning_rate=1.1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [11]:
model.fit_generator(training_set,
                    steps_per_epoch=training_set.samples//BATCH_SIZE,
                    validation_data=validation_set,
                    validation_steps=validation_set.samples//BATCH_SIZE,
                    epochs=EPOCHS)

Epoch 1/5
9537/9537 [==============================] - 8149s 855ms/step - loss: 4.2726 - accuracy: 0.1960 - val_loss: 4.7625 - val_accuracy: 0.2061
Epoch 2/5
9537/9537 [==============================] - 7859s 824ms/step - loss: 3.1924 - accuracy: 0.3282 - val_loss: 3.7884 - val_accuracy: 0.2557
Epoch 3/5
9537/9537 [==============================] - 7764s 814ms/step - loss: 2.7871 - accuracy: 0.3930 - val_loss: 2.4607 - val_accuracy: 0.2845
Epoch 4/5
9537/9537 [==============================] - 7771s 815ms/step - loss: 2.5248 - accuracy: 0.4351 - val_loss: 3.2217 - val_accuracy: 0.3120
Epoch 5/5
9537/9537 [==============================] - 7784s 816ms/step - loss: 2.3316 - accuracy: 0.4694 - val_loss: 4.2263 - val_accuracy: 0.3293


In [12]:
model.save('../Models/vgg19.h5')

In [13]:
with open('../Mappings/class_mappings_vgg_19.json', 'w', encoding='utf-8') as f:
    json.dump(validation_set.class_indices, f, ensure_ascii=False, indent=4)